In [1]:
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '\
# --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.2.0,com.datastax.spark:spark-cassandra-connector_2.11:2.0.1 \
# pyspark-shell'

In [2]:
# #    Spark
# from pyspark import SparkConf, SparkContext
# #    Spark Streaming
# from pyspark.streaming import StreamingContext
# #    Kafka
# from pyspark.streaming.kafka import KafkaUtils

# from pyspark.sql import SQLContext

# #    json parsing
# import json

In [3]:
# conf = SparkConf(True).setMaster("local[*]").setAppName("jupyter pyspark").set("spark.cassandra.connection.host", "cassandra")
# sc = SparkContext(conf=conf)
# #sc = SparkContext(appName= "jupyter pyspark")

# sc.setLogLevel("WARN")
# sqlContext = SQLContext(sc)

In [4]:
#sc.stop()

In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import pandas.tseries.offsets as offsets
%matplotlib inline

In [6]:
sf_data = pd.read_csv("./raw_data/sanfrancisco.csv")

In [7]:
raw_columns = list(sf_data)
raw_columns

['dt',
 'dt_iso',
 'city_id',
 'city_name',
 'lat',
 'lon',
 'temp',
 'temp_min',
 'temp_max',
 'pressure',
 'sea_level',
 'grnd_level',
 'humidity',
 'wind_speed',
 'wind_deg',
 'rain_1h',
 'rain_3h',
 'rain_24h',
 'rain_today',
 'snow_1h',
 'snow_3h',
 'snow_24h',
 'snow_today',
 'clouds_all',
 'weather_id',
 'weather_main',
 'weather_description',
 'weather_icon']

In [8]:
unused_columns = ['snow_1h', 'snow_24h', 'rain_24h', 'rain_1h', 'snow_3h', 'rain_today', 'snow_today', 'weather_icon', 'sea_level', 'grnd_level', 'lat', 'lon', 'city_id', 'city_name']

In [9]:
columns = list(set(raw_columns) - set(unused_columns))

In [10]:
sf_data2 = sf_data[columns]

In [11]:
sf_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38415 entries, 0 to 38414
Data columns (total 14 columns):
weather_id             38415 non-null int64
wind_speed             38415 non-null int64
dt                     38415 non-null int64
rain_3h                1355 non-null float64
temp_min               38415 non-null float64
wind_deg               38415 non-null int64
clouds_all             38415 non-null int64
dt_iso                 38415 non-null object
temp_max               38415 non-null float64
pressure               38415 non-null int64
humidity               38415 non-null int64
weather_main           38415 non-null object
temp                   38415 non-null float64
weather_description    38415 non-null object
dtypes: float64(4), int64(7), object(3)
memory usage: 4.1+ MB


In [12]:
sf_data2['rain_3h']= sf_data2['rain_3h'].fillna(0)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
sf_data2.drop_duplicates('dt', inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/util/decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [14]:
sf_data2.info()pd.to_datetime

SyntaxError: invalid syntax (<ipython-input-14-52c72f38c80b>, line 1)

In [26]:
sf_data2['dt_datetime'] =  pd.to_datetime(sf_data2['dt_iso'], format='%Y-%m-%d %H:%M:%S +%f %Z')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [141]:
sf_data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33442 entries, 0 to 38414
Data columns (total 15 columns):
weather_id             33442 non-null int64
wind_speed             33442 non-null int64
dt                     33442 non-null int64
rain_3h                33442 non-null float64
temp_min               33442 non-null float64
wind_deg               33442 non-null int64
clouds_all             33442 non-null int64
dt_iso                 33442 non-null object
temp_max               33442 non-null float64
pressure               33442 non-null int64
humidity               33442 non-null int64
weather_main           33442 non-null object
temp                   33442 non-null float64
weather_description    33442 non-null object
dt_datetime            33442 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(7), object(3)
memory usage: 4.1+ MB


In [42]:
int(pd.Timedelta(sf_data2['dt_datetime'].iloc[2] - sf_data2['dt_datetime'].iloc[1]).seconds/3600)

1

In [177]:
diff_columns = list(set(sf_data2.columns) - set(['dt', 'dt_iso']))
diff_columns
day_diff = 2
maximum_prev_index_diff = 2 *  24 
raw_datum = sf_data2.reset_index(drop=True).iloc[0:100]
raw_datum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
weather_id             100 non-null int64
wind_speed             100 non-null int64
dt                     100 non-null int64
rain_3h                100 non-null float64
temp_min               100 non-null float64
wind_deg               100 non-null int64
clouds_all             100 non-null int64
dt_iso                 100 non-null object
temp_max               100 non-null float64
pressure               100 non-null int64
humidity               100 non-null int64
weather_main           100 non-null object
temp                   100 non-null float64
weather_description    100 non-null object
dt_datetime            100 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(7), object(3)
memory usage: 11.8+ KB


In [178]:
def add_diff_datum():
    data = raw_datum.copy()
    data = data.apply(add_diff_days_data, axis=1)
    return data
sf_data3 = add_diff_datum()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [180]:
sf_data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 314 entries, clouds_all to wind_speed_9_ago
dtypes: datetime64[ns](1), float64(234), int64(7), object(72)
memory usage: 245.4+ KB


In [165]:
def add_diff_days_data(current_data):    
    index = current_data.name
    print(index)

    if index == 0:
        return current_data
    prev_index_right = index - 1
    left_index = index - maximum_prev_index_diff if index - maximum_prev_index_diff > 0 else 0
       
    while left_index <= prev_index_right:
        current_data = add_diff_data(current_data, raw_datum.iloc[left_index])
        left_index += 1
    return current_data

In [159]:
def add_diff_data(current_data, prev_data):
    diff = int(pd.Timedelta(current_data['dt_datetime'] - prev_data['dt_datetime']).seconds/ 3600)
    if diff > 0 and diff < maximum_prev_index_diff:
        for diff_column in diff_columns:
            column_name = '{}_{}_ago'.format(diff_column, diff)
            current_data[column_name] = prev_data[diff_column]
    return current_data

In [75]:
# def add_days_diff(raw_data, diff_columns):
#     data = raw_data.copy()
#     maximum_prev_index_diff = day_diff * 24
#     for index, current_data in data.iterrows():
#         prev_index_right = index -1
#         #maximum_prev_index = index - maximum_prev_index_diff
#         prev_index_left = if index - maximum_prev_index_diff > 0:
#             index - maximum_prev_index_diff
#             else:
#                 0
#         prev_datum  = raw_data[prev_index_left:prev_index_right]
        
#         while prev_index >= 0 and prev_index > maximum_prev_index:
#             print('current_index{}_:prev_index:{}'.format(index, prev_index))
#             add_diff_data(day_diff, current_data, data.iloc[prev_index], diff_columns)
#             prev_index = prev_index - 1
#         else:
#             print("loop is done from {} to {}".format(index, prev_index))
        
#     return data

In [ ]:
#create 2 d diff data
# change category data to numercial dataa
# explore data
# apply machine learning algorithm